In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 797155586758859868, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9221160305
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6931650197846934814
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6683898676
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4390883013433666553
 physical_device_desc: "device: 1, name: GeForce GTX 1070 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"]

In [5]:
import os
import numpy as np
np.random.seed(777)

from keras.models import Model,load_model
from keras import Input
from keras.layers import concatenate,Dense,Flatten,Dropout
from keras.preprocessing.image import  ImageDataGenerator
import keras.backend as K

from keras.utils import plot_model

from keras.optimizers import SGD, Adam, RMSprop

In [6]:
import keras
print("Keras Version", keras.__version__)

import tensorflow as tf

print("tensorflow Version", tf.__version__)


print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.1.6
tensorflow Version 1.10.0
dim_ordering: tf


In [7]:
output_classes = 2
batch_size = 8
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 59168
nb_validation_samples = 18384
nb_test_samples = 967

In [8]:
train_dir = 'data/train/'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [9]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 59168 images belonging to 2 classes.
Found 18384 images belonging to 2 classes.
Found 967 images belonging to 2 classes.


In [10]:
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)
sgd_opt = SGD(lr=1e-06, momentum=0.0, decay=0.0, nesterov=False)
rmsp_opt = RMSprop(lr=1e-4, decay=0.9)

In [11]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [12]:
# NasNetLarge_model = load_model('models/5.NASNetLarge-ISBI19-DataSet-Model.h5')
# NasNetLarge_model = Model(inputs= NasNetLarge_model.input,outputs = NasNetLarge_model.get_layer('dense_2').output, name='NasNetLarge_model')

In [13]:
# InceptionResNetV2_model = load_model('models/5.InceptionResNetV2-ISBI19-Model.h5')
# InceptionResNetV2_model = Model(inputs=resnet_model.input,outputs=resnet_model.get_layer('dense_2').output,name='InceptionResNetV2_model')

In [14]:
DenseNet201_model = load_model('models/12.DenseNet201-ISIB19-DataSet-Model.h5')
DenseNet201_model = Model(inputs=DenseNet201_model.input, outputs=DenseNet201_model.get_layer('dense_3').output, name='DenseNet201_model')

In [15]:
InceptionV3_model = load_model('models/9.InceptionV3-ISBI19-DataSet-Model.h5')
InceptionV3_model = Model(inputs = InceptionV3_model.input,outputs = InceptionV3_model.get_layer('dense_3').output, name='InceptionV3_model')

In [16]:
VGG16_model = load_model('models/13.VGG16-ISIB19-Model.h5')
VGG16_model = Model(inputs = VGG16_model.input,outputs = VGG16_model.get_layer('dense_8').output, name='VGG16_model')

In [17]:
# def ensemble(input_shape, num_class, epochs, savepath = './ensemble.h5'):
    
#     img = Input(shape=(224, 224,3),name='img')
    
#     feature1=DenseNet201_model(img)
#     feature2=InceptionV3_model(img)
    
#     for layer in InceptionV3_model.layers[:300]:  
#         layer.trainable = False 
#     for layer in InceptionV3_model.layers[300:]:  
#         layer.trainable = True  
    
#     feature3=VGG16_model(img)
#     for layer in VGG16_model.layers[:170]:  
#         layer.trainable = False 
#     for layer in VGG16_model.layers[170:]:  
#         layer.trainable = True  
    
    
#     x = concatenate([feature1,feature2,feature3])
    
#     x = Dropout(0.5)(x)
#     x = Dense(64,activation='relu')(x)
#     x = Dropout(0.25)(x)
#     output = Dense(2, activation='softmax', name='output')(x)
    
    
#     model=Model(inputs=img,outputs=output)
    
# #     plot_model(model=model,to_file='./ensemble.jpg')
    
#     adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)
#     sgd_opt = SGD(lr=1e-06, momentum=0.0, decay=0.0, nesterov=False)
#     rmsp_opt = RMSprop(lr=1e-4, decay=0.9)
#     opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#     model.compile(loss='categorical_crossentropy',\
# #                   optimizer=SGD(lr=0.001, momentum=0.9),
#                   optimizer=adam_opt,
#                   metrics=['accuracy'])

#     hist = model.fit_generator(
#     train_generator,
#     steps_per_epoch=nb_train_samples // batch_size,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=nb_validation_samples // batch_size,
#     workers=8
#     )
#     with open('ensemble.txt','w') as f:
#         f.write(str(hist.history))

In [ ]:
# ensemble_model = ensemble(input_shape, num_class = 2, epochs = 50)

Epoch 1/50
7396/7396 [==============================] - 2556s 346ms/step - loss: 0.5074 - acc: 0.8424 - val_loss: 0.3281 - val_acc: 0.9556
Epoch 2/50
7396/7396 [==============================] - 2495s 337ms/step - loss: 0.2864 - acc: 0.9415 - val_loss: 0.2323 - val_acc: 0.9562
Epoch 3/50
7396/7396 [==============================] - 2492s 337ms/step - loss: 0.2171 - acc: 0.9505 - val_loss: 0.2031 - val_acc: 0.9606
Epoch 4/50
7396/7396 [==============================] - 2495s 337ms/step - loss: 0.1968 - acc: 0.9493 - val_loss: 0.2022 - val_acc: 0.9615
Epoch 5/50
7396/7396 [==============================] - 2493s 337ms/step - loss: 0.1854 - acc: 0.9511 - val_loss: 0.2025 - val_acc: 0.9611
Epoch 6/50
7396/7396 [==============================] - 2500s 338ms/step - loss: 0.1791 - acc: 0.9512 - val_loss: 0.2043 - val_acc: 0.9607
Epoch 7/50
5984/7396 [=======================>......] - ETA: 7:01 - loss: 0.1730 - acc: 0.9528

In [17]:
img = Input(shape=(224, 224,3),name='img')

feature1=DenseNet201_model(img)
feature2=InceptionV3_model(img)

for layer in InceptionV3_model.layers[:300]:  
    layer.trainable = False 
for layer in InceptionV3_model.layers[300:]:  
    layer.trainable = True  

feature3=VGG16_model(img)
for layer in VGG16_model.layers[:170]:  
    layer.trainable = False 
for layer in VGG16_model.layers[170:]:  
    layer.trainable = True  

x = concatenate([feature1,feature2,feature3])

x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.25)(x)
output = Dense(2, activation='softmax', name='output')(x)

model=Model(inputs=img,outputs=output)

In [ ]:
#     plot_model(model=model,to_file='./ensemble.jpg')

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

history = model.fit_generator(
train_generator,
steps_per_epoch=nb_train_samples // batch_size,
epochs=epochs,
validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size,
workers=8
)
with open('ensemble.txt','w') as f:
    f.write(str(hist.history))

Epoch 1/50
